In [9]:
!mkdir model_dir
!wget -O lmodel.zip https://github.com/natashadsilva/scratch/blob/master/LinearModel.zip?raw=true
!unzip -d model_dir  lmodel.zip

mkdir: cannot create directory ‘model_dir’: File exists
--2019-11-01 05:20:01--  https://github.com/natashadsilva/scratch/blob/master/LinearModel.zip?raw=true
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/natashadsilva/scratch/raw/master/LinearModel.zip [following]
--2019-11-01 05:20:01--  https://github.com/natashadsilva/scratch/raw/master/LinearModel.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/natashadsilva/scratch/master/LinearModel.zip [following]
--2019-11-01 05:20:01--  https://raw.githubusercontent.com/natashadsilva/scratch/master/LinearModel.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.36.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.36.133|:443... connected.
HT

In [12]:
import getpass
credentials=getpass.getpass('Streaming Analytics credentials:')

metadata  stages


In [32]:
import os
 
dirpath = os.getcwd()
model_path = "model_dir"

In [43]:
import streamsx.ec
import time
import random
from pyspark.ml import PipelineModel
from pyspark.sql import SparkSession
import warnings
import logging
import csv 
warnings.filterwarnings('ignore')


class LoadSparkModel:
    def __init__(self, model_file):
        self.model_path = model_path
    def __exit__(self, exc_type, exc_value, traceback):
        pass
    def __enter__(self):
        # Application is starting on the Streams runtime,
        
        logging.getLogger("SparkModel").info("INFO: Loading Spark Model")
        # load json and create model
        try :
            self.sparkSession = SparkSession.builder.appName("Score Spark Model with Streams").getOrCreate()
            #get path to model at runtime
            path_to_model = streamsx.ec.get_application_directory() + "/etc/" +  self.model_path
            self.sparkModel = PipelineModel.load(path_to_model)
            logging.getLogger("SparkModel").info("INFO: Successfully loaded ,odel")
        except Exception as e:
            logging.getLogger("SparkModel").error("ERROR loading file " +  str(e))
        
    def __call__(self, tpl):
        logging.getLogger("SparkModel").info("INFO: Going to run model on tuple")
        tpl_as_DF = self.sparkSession.createDataFrame([tpl])
        result = self.sparkModel.transform(tpl_as_DF).collect()[0]
        logging.getLogger("SparkModel").debug("INFO: Ran model on tuple")
        tpl["prediction"] = result.prediction
        return tpl


In [44]:
#create topology...change the name to something more meaningful
from streamsx.topology.topology import Topology
import streamsx.topology.context
import numpy as np
from pyspark.ml.regression import LinearRegression

topo = Topology(name="SparkScoring")

topo.add_file_dependency(model_path + "/", "etc")

# add stream...delete and replace this with your streams and analytics
# this is added so the topology will run (needs a source)
# produces a random integer between 1 and 100 every tenth of a second

import random, time

def readings():
    while True:
        time.sleep(0.1)
        time_now = 1541019341*1000
        record_processed = 0
        deviceID = np.random.randint(1, 3)
        sensorID = np.random.randint(1, 50)
        env_temp = np.random.normal(24.5, 2)  # ambient temp 
        time_now += np.random.randint(10,1500)
        power = np.random.normal(10, 3) # power consumption
        noise = np.random.normal(0,1.5)
        temp = 1.3 * env_temp + 0.5 * power + 5 + noise

        yield dict(device=int(deviceID), 
                   sensor=int(sensorID), ts=time_now, 
                   env_temp=float(env_temp),
                   power=float(power), 
                   temperature=float(temp))


source = topo.source(readings)
predictions = source.isolate().map(LoadSparkModel(model_path))
predictions.print()
pv = predictions.view()
sv = source.view
topo.add_pip_package("pyspark")
source.print()
# create a view to watch the stream data while running


In [47]:


from streamsx.topology import topology, context
import json

def submit_from_notebook(topo, service_instance_name, credentials={}, verify_ssl=False):
    isCloudPrivate = True
    cfg = {}
    try:
        from icpd_core import icpd_util
        cfg = icpd_util.get_service_instance_details(name=service_instance_name)
        contextType = context.ContextTypes.DISTRIBUTED
    except ImportError as error:
        #Cloud Pak API unavailable
        vs={'streaming-analytics': [{'name': service_name, 'credentials': json.loads (credentials)}]}
        cfg = {}
        cfg[ConfigParams.VCAP_SERVICES] = vs
        cfg[ConfigParams.SERVICE_NAME] = service_name
        contextType = context.ContextTypes.STREAMING_ANALYTICS_SERVICE
        
    cfg[context.ConfigParams.SSL_VERIFY] = verify_ssl
    job = context.submit(contextType, topo, config=cfg)
    return job

    



JobId:  0


In [55]:
#Submit

job = submit_from_notebook(topo, service_name, credentials)

if (job.id ):
    print("Job with id " + job.jobId  + " was submitted")
else:
    print ("Error submitting job")


AttributeError: 'function' object has no attribute 'fetch_tuples'